In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import tensorflow as tf
import numpy
import os
import cv2 as cv
from sklearn.model_selection import train_test_split

    
def new_conv_layer(inputd, channels, filters, filter_shape, pool_size, stride, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filter_shape = [filter_shape[0], filter_shape[1], channels, filters]
    
    #declaring pooling
    k_size = [1, pool_size[0], pool_size[1], 1]
    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.03), name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([filters]), name=name+'_b')

    # setup the convolutional layer operation
    outputlayer = tf.nn.conv2d(inputd, weights, [1, 1, 1, 1], padding='SAME')

    # add the bias
    outputlayer += bias

    # apply a ReLU non-linear activation
    outputlayer = tf.nn.relu(outputlayer)

    # now perform max pooling
    # ksize is the argument which defines the size of the max pooling window (i.e. the area over which the maximum is
    # calculated).  It must be 4D to match the convolution - in this case, for each image we want to use a 2 x 2 area
    # applied to each channel
    
    output_layer = tf.nn.max_pool(outputlayer, ksize=k_size, strides=stride, padding='SAME')

    return output_layer
  
# Python optimisation variables hyper-parameters
learning_rate = 0.0005
epochs = 12
batch_size = 60
  
#sparse array of training example classes 911*7
labels_train=[]

#images 911*224*224*3
images_train=[]
  
for path, directories, files in os.walk("drive/My Drive/veda/HumanActionClassification"):
  for i in directories:
    r=[0,0,0,0,0,0,0]
    r[int(i)-1]=1
    for subpath,subdirs,subfiles in os.walk("drive/My Drive/veda/HumanActionClassification/"+i):
      for file in subfiles:
        im=cv.resize(cv.imread("drive/My Drive/veda/HumanActionClassification/"+i+'/'+file),dsize=(224,224),interpolation=cv.INTER_CUBIC)
        images_train.append(im)
        labels_train.append(r)
train_images=numpy.array(images_train)
train_labels=numpy.array(labels_train)
#splitting
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.30)


# declare the training data placeholders
x = tf.placeholder(tf.float32, [None, 224,224,3])
y = tf.placeholder(tf.float32, [None, 7])

# create some convolutional layers
layer_1 = new_conv_layer(x, 3, 32, [3, 3], [3, 3], [1, 2, 2, 1], name='layer1')
layer_2 = new_conv_layer(layer_1, 32, 64, [5, 5], [5, 5], [1, 2, 2, 1], name='layer2')
layer_3 = new_conv_layer(layer_2, 64, 128, [7, 7], [9, 9], [1, 8, 8, 1], name='layer3')
flatten = tf.reshape(layer_3, [-1, 7 * 7 * 128])

# setup some weights and bias values for this layer, then activate with ReLU
# another layer with softmax activations
#weights of dense layer
wd2 = tf.Variable(tf.truncated_normal([7*7*128, 7], stddev=0.03), name='wd2')
#bias of dense layer
bd2 = tf.Variable(tf.truncated_normal([7], stddev=0.01), name='bd2')
    
denselayer2 = tf.matmul(flatten, wd2) + bd2
y_ = tf.nn.softmax(denselayer2)
#loss function
Cross_Entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=denselayer2, labels=y))

# add an optimiser
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(Cross_Entropy)

# define an accuracy assessment operation
accurate_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(accurate_prediction, tf.float32))

# setup the initialisation operator
init_op_ = tf.global_variables_initializer()

# setup recording variables
# add a summary to store the accuracy
  
with tf.Session() as s:
    # initialise the variables
    s.run(init_op_)
    no_of_batch = int( X_train.shape[0]/ batch_size)                    
    for epoch in range(1,epochs+1):
        avg_cost = 0
        for i in range(no_of_batch):
            #print(epoch,i)
            x_batch = X_train[i*batch_size:(i+1)*batch_size]
            y_batch = y_train[i*batch_size:(i+1)*batch_size]
            _, c = s.run([optimiser, Cross_Entropy], feed_dict={x: x_batch, y: y_batch})
            avg_cost += c / no_of_batch
        test_accuracy = s.run(accuracy, feed_dict={x: X_test, y: y_test})
        print("Epoch no:", epoch)
        print("cost =", "{:.2f}".format(avg_cost),"||", " test accuracy: {:.2f}".format(test_accuracy))
        print("---------------------------")
    print("\nTraining complete!")
    print(s.run(accuracy, feed_dict={x: X_test, y: y_test}))


Epoch no: 1
cost = 36.01 ||  test accuracy: 0.17
---------------------------
Epoch no: 2
cost = 4.19 ||  test accuracy: 0.22
---------------------------
Epoch no: 3
cost = 2.03 ||  test accuracy: 0.22
---------------------------
Epoch no: 4
cost = 1.83 ||  test accuracy: 0.27
---------------------------
Epoch no: 5
cost = 1.70 ||  test accuracy: 0.30
---------------------------
Epoch no: 6
cost = 1.60 ||  test accuracy: 0.30
---------------------------
Epoch no: 7
cost = 1.48 ||  test accuracy: 0.30
---------------------------
Epoch no: 8
cost = 1.36 ||  test accuracy: 0.31
---------------------------
Epoch no: 9
cost = 1.24 ||  test accuracy: 0.34
---------------------------
Epoch no: 10
cost = 1.10 ||  test accuracy: 0.32
---------------------------
Epoch no: 11
cost = 0.99 ||  test accuracy: 0.35
---------------------------
Epoch no: 12
cost = 0.89 ||  test accuracy: 0.34
---------------------------

Training complete!
0.33941606
